In [1]:
import pandas as pd
import torch

In [3]:
df = pd.read_csv('../datasets/imagenet_label_map.csv')
df

,index,label,annotation
0,0,n02389026,horse
1,1,n03888257,parachute
2,2,n03584829,iron
3,3,n02607072,clownfish
4,4,n03297495,coffee maker
5,5,n03063599,coffee cup
6,6,n03792782,bicycle
7,7,n04086273,gun
8,8,n02510455,panda
9,9,n11939491,flower


In [67]:
data = torch.load('../datasets/eeg_5_95_std.pth')
data.keys()

dict_keys(['dataset', 'labels', 'images'])

In [68]:
subset_classes = [0, 1, 5, 6, 7, 9, 10, 11, 12, 28, 29, 37]
subset_labels = ['n02389026', 'n03888257', 'n03063599', 'n03792782', 'n04086273', 'n11939491', 'n02951358', 'n02281787', 'n02106662', 'n03272010', 'n04069434', 'n03272562']

In [69]:
data_subset = {
    'dataset': [],
    'labels': subset_labels,
    'images': []
}
for sample in data['dataset']:
    if sample['label'] in subset_classes and sample['subject'] == 4:
        data_subset['images'].append(data['images'][sample['image']])
        sub_sample = {
            'eeg': sample['eeg'],
            'image': len(data_subset['images']) - 1,
            'label': subset_labels.index(data['labels'][sample['label']]),
            'subject': sample['subject']
        }
        data_subset['dataset'].append(sub_sample)

In [70]:
torch.save(data_subset, '../datasets/eeg_subset.pth')

In [71]:
data_subset = torch.load('../datasets/eeg_subset.pth')
data_subset

{'dataset': [{'eeg': tensor([[-0.0098,  0.0195,  0.0620,  ...,  0.0638,  0.0120, -0.0118],
           [-0.0045,  0.1303,  0.2673,  ...,  0.0894,  0.0342, -0.0082],
           [ 0.0215, -0.2017, -0.4305,  ..., -0.2022, -0.0940,  0.0188],
           ...,
           [ 0.0160,  0.0707,  0.1005,  ...,  0.2066,  0.1156,  0.0036],
           [-0.0046, -0.0084, -0.0119,  ...,  0.0007, -0.0026, -0.0053],
           [ 0.0040,  0.0419,  0.0665,  ...,  0.0765,  0.0309, -0.0063]]),
   'image': 0,
   'label': 6,
   'subject': 4},
  {'eeg': tensor([[-0.0120,  0.0518,  0.1203,  ..., -0.1646, -0.0644,  0.0195],
           [-0.0165,  0.0830,  0.1887,  ..., -0.2273, -0.0995,  0.0131],
           [ 0.0161, -0.0758, -0.1795,  ..., -0.0401, -0.0406, -0.0183],
           ...,
           [-0.0101,  0.0029, -0.0043,  ...,  0.1310,  0.0679, -0.0143],
           [-0.0060, -0.0082, -0.0111,  ...,  0.0098,  0.0033, -0.0056],
           [-0.0132, -0.0258, -0.0471,  ...,  0.1730,  0.0961, -0.0068]]),
   'image': 1,


In [72]:
len(data_subset['dataset'])

600

In [74]:
splits_subset = {'splits': [{'train': [], 'val': [], 'test': []}]}
splits_subset['splits'][0].keys()

dict_keys(['train', 'val', 'test'])

In [76]:
label_frequencies = {} # 50 instances for 12 classes
for idx, sample in enumerate(data_subset['dataset']):
    try:
        label_frequencies[sample['label']] += 1
    except KeyError:
        label_frequencies[sample['label']] = 1
    split_type = 'test' if label_frequencies[sample['label']] > 40 else 'train'
    splits_subset['splits'][0][split_type].append(idx)

In [82]:
len(splits_subset['splits'][0]['train'])

480

In [81]:
len(splits_subset['splits'][0]['test'])

120

In [84]:
torch.save(splits_subset, '../datasets/block_splits_subset.pth')

In [85]:
splits_subset = torch.load('../datasets/block_splits_subset.pth')
splits_subset['splits'][0].keys()

dict_keys(['train', 'val', 'test'])

In [86]:
train_len, val_len, test_len = len(splits_subset['splits'][0]['train']), len(splits_subset['splits'][0]['val']), len(splits_subset['splits'][0]['test'])
print(f'{train_len} + {val_len} + {test_len} = {train_len + val_len + test_len}')

480 + 0 + 120 = 600
